In [153]:
import pandas as pd
import random

In [4]:
%run "card_loader.ipynb"

In [155]:
def from_edition(cards, edition):
    return cards["setCode"] == edition
    #return cards["printings"].apply(lambda printings: edition in printings)

def from_rarity(cards, rarity):
    return cards["rarity"] == rarity

def not_basic(cards):
    basic = ["Island", "Plains", "Swamp", "Mountain", "Forest"]
    return cards.name.apply(lambda name: name not in basic) 

def only_color(cards, color):
    return cards.colors == color

def is_saga(cards):
    return cards.type.apply(lambda x : "Saga" in x)
    
def from_type(cards, card_type):
    return cards.type == card_type

In [178]:
def find_unique(subset):
    #print(set(subset.name))
    subset = subset[subset.name.apply(lambda x: "A-" not in x)]
    ids = [subset[subset.name_short == name].iloc[0].id for name in set(subset.name_short)]
    return subset.loc[subset['id'].isin(ids)]

In [176]:
def booster_neo():
    relevant = cards[from_edition(cards, "NEO") & not_basic(cards)]
    uncommons = find_unique(relevant[from_rarity(relevant, "uncommon") & (~is_saga(relevant))])
    rares = find_unique(relevant[from_rarity(relevant, "rare")])
    mythics = find_unique(relevant[from_rarity(relevant, "mythic")])
    
    sagas = find_unique(relevant[is_saga(relevant)  & (from_rarity(relevant, "common") | from_rarity(relevant, "uncommon"))])
    
    commons = find_unique(relevant[from_rarity(relevant, "common") & (~from_type(relevant, "Land"))])
    lands = find_unique(relevant[from_rarity(relevant, "common") & from_type(relevant, "Land") & (~is_saga(relevant))])
    commons_red = find_unique(relevant[from_rarity(relevant, "common") & only_color(relevant, "R") & (~is_saga(relevant))])
    commons_white = find_unique(relevant[from_rarity(relevant, "common") & only_color(relevant, "W") & (~is_saga(relevant))])
    commons_green = find_unique(relevant[from_rarity(relevant, "common") & only_color(relevant, "G") & (~is_saga(relevant))])
    commons_black = find_unique(relevant[from_rarity(relevant, "common") & only_color(relevant, "B") & (~is_saga(relevant))])
    commons_blue = find_unique(relevant[from_rarity(relevant, "common") & only_color(relevant, "U") & (~is_saga(relevant))])
    
    name_rare = rares.sample(1).name_short
    if random.random() < 1/7:
        name_rare = mythics.sample(1).name_short
        
    name_uncommon = uncommons.sample(3).name_short
    name_saga = sagas.sample(1).name_short
    
    name_red = commons_red.sample(1).name_short
    name_white = commons_white.sample(1).name_short
    name_green = commons_green.sample(1).name_short
    name_black = commons_black.sample(1).name_short
    name_blue = commons_blue.sample(1).name_short
    
    used_names = list(name_red) + list(name_white) + list(name_green) + list(name_black) + list(name_blue)
    commons = commons[commons.name.apply(lambda x: x not in used_names)]
    
    name_common = commons.sample(4).name_short
    name_land = lands.sample(1).name_short
    
    return list(name_rare) + list(name_uncommon) + list(name_saga)+list(name_red)+list(name_white) + \
          list(name_green) + list(name_black) +list(name_blue) +list(name_common) +list(name_land)